In [ ]:
import numpy as np
import plotly.graph_objects as go
def save_figure_to_file(fig, path, file_name):
    fig.write_image(path + '/' + file_name + '.svg')

In [ ]:
n = 4

Z = np.random.standard_normal((n,n))

Q, R = np.linalg.qr(Z,mode='complete')

# test
np.testing.assert_array_less(np.eye(n) - np.dot(Q,Q.T), 1e-14)

d = 0.1 + np.random.random((n,))
d = np.sort(d)
D = np.diag(d)

H = np.dot(Q, np.dot(D,Q.T))
U, S, VT = np.linalg.svd(H, full_matrices=True)

# test symmetry
np.linalg.norm(H - H.T)

# test evals
w, v = np.linalg.eig(H)
w = np.sort(w)
np.testing.assert_array_less(w-d, 1e-14)

beta = 0.1

D = np.diag([beta**k for k in range(n)])

Hb = np.dot(D, np.dot(H, D))

U, S, VT = np.linalg.svd(Hb, full_matrices=True)

for i in range(n):
    if U[i,i] < 0:
        U[:,i] = -U[:,i]
        VT[i,:] = -VT[i,:]

print('Hb=')
print(Hb)
print('U=')
print(U)
print('V=')
print(VT.T)
print('Sigma=')
print(S/S[0])

In [ ]:
x0 = np.random.standard_normal((n,1))

D = np.diag([beta**(-k) for k in range(n)])

alpha = 1

w, v = np.linalg.eig( alpha * Hb )
print(w)

w, v = np.linalg.eig( alpha * np.dot(D, Hb) )
print(w)

x = x0.copy()
print('|x0|=',np.linalg.norm((x)))

k_iter = 10000
print('number of iterations=',k_iter)
G = np.eye(n) - alpha * np.dot(D, Hb)
norm_x_0 = np.zeros(1 + int((k_iter-1)/100))
for k in range(k_iter):
    x = np.dot(G, x)
    if k%100 == 0:
        norm_x_0[int(k/100)] = np.linalg.norm((x))

print('x=',np.linalg.norm((x)))
        
x = x0.copy()
k_iter = int(k_iter/beta**(n-1))
print('number of iterations=',k_iter)
G = np.eye(n) - alpha * Hb
norm_x_1 = np.zeros(1 + int((k_iter-1)/100))
for k in range(k_iter):
    x = np.dot(G, x)
    if k%100 == 0:
        norm_x_1[int(k/100)] = np.linalg.norm((x))     

print('x=',np.linalg.norm((x)))

In [ ]:
beta = 0.1
n = 4
# beta = 0.9
# n = 100
m = n-1
print('Estimated condition number: {:.2e}\n'.format(beta**(2*m)))
print('Estimated number of iterations with Adagrad: {:.2e}\n'.format(beta**(-m)))
print('Estimated number of iterations without Adagrad: {:.2e}\n'.format(beta**(-2*m)))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=norm_x_1,name='Gradient Descent'))
fig.add_trace(go.Scatter(y=norm_x_0,name='Adagrad'))
fig.update_layout(height=400, width=600, xaxis_type="log", yaxis_type="log", 
                  xaxis_title='iteration', yaxis_title='error')
fig.show()

In [ ]:
save_figure_to_file(fig,'../Slides/Deep_Learning','fig6')